In [1215]:
%run '07_Lefschetz_Collection.ipynb'

In [1301]:
class Young_Structure ( object ) :

    def __eq__ ( self , other ) -> bool :
        if isinstance( other , self.__class__ ) : return self.Remove_Vanishing_Values().__repr__() == other.Remove_Vanishing_Values().__repr__()
        else                                    : return False


    def __getitem__ ( self , Coordinate :tuple[ int ] ) -> int :
        """Returns the value of a given coordinate."""
        assert isinstance( Coordinate , tuple ) , \
               'The input for `Coordinate` need to be a tuple.'
        assert len(Coordinate) == self.Rank() , \
               'The length of `Coordinate` need to coincide with the rank of `self`.'
        for CoordinateCounter , CoordinateEntry in enumerate( Coordinate , start=1 ) :
            assert isinstance( CoordinateEntry , ( int , Integer ) ) , \
                   'The '+str(CoordinateCounter)+'-th entry of the coordinate need to be an integer.'
        if Coordinate in self.Data().keys() : return self.Data()[Coordinate]
        else                                : return 0


    def __ge__ ( self , other ) -> bool :
        """Compares Young structures with respect to the >=-operator."""
        return other <= self


    def __gt__ ( self , other ) -> bool :
        """Compares Young structures with respect to the >-operator."""
        return other < self


    def __init__( self , Data :dict , Ranges :list ) -> None :
        """
        Initialize a Young structure.

        INPUT:
        - ``Data`` -- Dictionary ;
        - ``Ranges`` -- List of 2-tuples ;

        OUTPUT: None.
        """
        assert isinstance( Ranges , list ) , \
               'The input for `Ranges` need to be a list.'
        for RangeCounter , Entry in enumerate( Ranges , start=1 ) :
            assert isinstance( Entry , tuple ) , \
                   'The '+str(RangeCounter)+'-th range need to be a tuple.'
            assert len(Entry) == 2 , \
                   'The '+str(RangeCounter)+'-th range need to be of length 2.'
            Minimum , Maximum = Entry
            assert isinstance( Minimum , ( int , Integer ) ) , \
                   'The minimum entry of the '+str(RangeCounter)+'-th range need to be an integer.'
            assert isinstance( Maximum , ( int , Integer ) ) , \
                   'The maximum entry of the '+str(RangeCounter)+'-th range need to be an integer.'
            assert Minimum <= Maximum , \
                   'For the '+str(RangeCounter)+'-th range, the minimum entry need to smaller or equal to the maximum entry.'
        self._Ranges = Ranges

        self._Rank = len(Ranges)

        assert isinstance( Data , dict ) , \
               'The input for `Data` need to be dictionary.'
        Adjusted_Data = dict({})
        for ValueCounter , ( Coordinate , Value ) in enumerate( Data.items() , start=1 ) :
            assert isinstance( Coordinate , tuple ) , \
                   'The '+str(ValueCounter)+'-th coordinates need to be a tuple.'
            assert len(Coordinate) <= self._Rank , \
                   'The '+str(ValueCounter)+'-th coordinates need to be of length <= '+str(self._Rank)+'.'
            Coordinate += tuple( ( self._Rank-len(Coordinate) )*[0] )
            if Coordinate in Adjusted_Data.keys() : raise ValueError('CONFLICT! There are two values associated the same coordinate (after adjusting).')
            else                                  : Adjusted_Data.update({ Coordinate : Value })
            for CoordinateCounter , CoordinateValue in enumerate( Coordinate ) :
                assert isinstance( CoordinateValue , ( Integer , int ) ) , \
                       'The '+str(CoordinateCounter+1)+'-th entry of the '+str(ValueCounter)+'-th coordinate need to be an integer.'
                Minimum = self._Ranges[CoordinateCounter][0]
                assert Minimum <= CoordinateValue , \
                       'The '+str(CoordinateCounter+1)+'-th entry of the '+str(ValueCounter)+'-th coordinate need to >= '+str(Minimum)+'.'
                Maximum = self._Ranges[CoordinateCounter][1]
                assert CoordinateValue <= Maximum , \
                       'The '+str(CoordinateCounter+1)+'-th entry of the '+str(ValueCounter)+'-th coordinate need to <= '+str(Maximum)+'.'
        self._Data = Adjusted_Data


    def __iter__ ( self ) -> Iterator[ tuple[ tuple[ int ] , int ] ] :
        """Returns an iterator of ``self``."""
        return ( ( Coordinate , Value ) for Coordinate , Value in self.Data().items() )


    def __le__ ( self , other ) -> bool :
        """Compares Young structures with respect to the <=-operator."""
        if   self == other : return True
        elif self <  other : return True
        else               : return False


    # Synonym for the method `Rank`
    def __len__( self ) :
        """Returns the rank ``self``."""
        return self.Rank()


    def __ne__ ( self , other ) -> bool :
        return not self == other


    def __repr__ ( self ) -> tuple[ dict[ tuple[int] : int ] , list[ tuple[ int , int ] ] ] :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Data() , self.Ranges()


    def __str__ ( self ) -> str :
        """Returns a description in terms of diagrams (sliced if necessary)."""
        if self.Rank() < 3 :
            # Low-dimensional Young structures can be printed.
            ( xMinimum , xMaximum ) , ( yMinimum , yMaximum ) = self.Ranges() + (2-self.Rank())*[ ( 0 , 0 ) ]
            Width  = xMaximum-xMinimum+1
            Height = yMaximum-yMinimum+1
            M = matrix( nrows=Height , ncols=Width )
            if 0 in [ yMinimum .. yMaximum ] : ySubdivision = [ abs(yMinimum) ]
            else                             : ySubdivision = None
            if 0 in [ xMinimum .. xMaximum ] : xSubdivision = [ abs(xMinimum) ]
            else                             : xSubdivision = None
            M.subdivide( ySubdivision , xSubdivision )
            for Coordinate , Value in self :
                xCoordinate , yCoordinate = Coordinate + tuple( (2-self.Rank())*[ 0 ] )
                M[ yCoordinate-yMinimum , xCoordinate-xMinimum ] = Value
            return str(M)

        else :
            # Slice down to the first two coordinates x and y.
            Output = ''
            x = self.Coordinates()
            Condition_Factors = []
            for CoordinateCounter in range( self.Rank()-1 , 1 , -1 ) :
                Minimum , Maximum = self.Ranges()[CoordinateCounter]
                Condition_Factors += [ [ x[CoordinateCounter] == Entry for Entry in [ Minimum .. Maximum ] ] ]
            for Conditions in cartesian_product(Condition_Factors) :
                YS = self.Locus( *Conditions )
                for CoordinateCounter in range( YS.Rank()-1 , 1 , -1 ) :
                    YS.Remove_Coordinate(CoordinateCounter)
                Output += str( Conditions ) + '\n'
                Output += str( YS ) + '\n'
                Output += '\n'

            return Output


    #ToDo: Test if `Permutation` is the correct instance.
    def Conjugate ( self , Permutation: "Permutation" ) -> "Young_Structure" :
        """Returns a conjugated Young structure."""
        assert len(Permutation) == self.Rank() , \
               'The length of the permutation need to coincide with the rank of `self`.'
        New_Data  = { tuple(Permutation.action(list(Coordinate))) : Value for Coordinate , Value in self }
        New_Ranges = Permutation.action(self.Ranges())
        return self.__class__( New_Data , New_Ranges )


    @staticmethod
    def Constructor ( *Input:tuple ) -> "Young_Structure" :

        def Get_Empty_Young_Structure ( Ranges :list[ tuple[ int , int ] ] ) -> "Young_Structure" :
            Data = { tuple(Coordinate) : 0 for Coordinate in cartesian_product([ [ Minimum .. Maximum ]
                                                                                 for Minimum , Maximum in Ranges
                                                                               ])
                   }
            return Young_Structure( Data , Ranges )

        def Get_Young_Structure_From_Binary_Description ( String :str ) -> "Young_Structure" :
            Keep_Running = True
            while Keep_Running :
                Keep_Running = False
                for SubString in [ 'RL' , 'LR' , 'DU' , 'UD' ] :
                    if String.find(SubString) != -1 :
                        String = String.replace( SubString , '' )
                        Keep_Running = True

            xAmplitude = [0]+[ String[:Counter].count('R')-String[:Counter].count('L') for Counter in range(len(String)) ]
            xRange = ( min(xAmplitude) , max(xAmplitude)-1 )
            if xRange == (0,-1) : xRange = (0,0)
            xMinimum , xMaximum = xRange
            yAmplitude = [0]+[ String[:Counter].count('D')-String[:Counter].count('U') for Counter in range(len(String)) ]
            yRange = ( min(yAmplitude) , max(yAmplitude)-1 )
            if yRange == (0,-1) : yRange = (0,0)
            yMinimum , yMaximum = yRange

            YS = Get_Empty_Young_Structure( [ xRange , yRange ] )
            Data = YS.Data()
            Ranges = YS.Ranges()

            xPointer , yPointer = (0,0)
            for Letter in String :
                if   Letter == 'R' :
                    for yPos in [ yPointer .. yMaximum ] :
                        Data[ (xPointer,yPos) ] += 1
                    xPointer += 1
                elif Letter == 'L' :
                    xPointer -= 1
                    for yPos in [ yPointer .. yMaximum ] :
                        Data[ (xPointer,yPos) ] -= 1
                elif Letter == 'D' :
                    yPointer += 1
                elif Letter == 'U' :
                    yPointer -= 1
            return Young_Structure( Data , Ranges )

        def Get_Young_Structure_From_Integer ( IntegerValue :int ) -> "Young_Structure" :
            if IntegerValue == 0 :
                Data = dict({})
                Ranges = [ ( 0 , 0 ) ]
            else :
                Minimum = min([ 0 , IntegerValue ])
                Maximum = max([ 0 , IntegerValue ])
                Data   = { (Index,) : 1 for Index in range(Minimum,Maximum) }
                Ranges = [ ( Minimum , Maximum-1 ) ]
            return Young_Structure( Data , Ranges )

        def Stacking_Young_Structures ( Stack :dict[ int : "Young_Structure" ] ) -> "Young_Structure" :
            if 0 < len( Stack.keys() ) :
                New_Rank = max([0] + [ New_Slice.Rank() for New_Slice in Stack.values() ]) + 1
                New_Data = dict({})
                New_Ranges = New_Rank * [ ( +infinity , -infinity ) ]
                for New_Coordinate , New_Slice in Stack.items() :
                    Filling = tuple( (New_Rank-New_Slice.Rank()-1)*[ 0 ])
                    New_Data.update({ Coordinate + Filling + (New_Coordinate,) : Value for Coordinate , Value in New_Slice })
                    Adjusted_Ranges = [ ( min([ New_Ranges[RangeCounter][0] , Range[0] ]) , max([ New_Ranges[RangeCounter][1] , Range[1] ]) ) for RangeCounter , Range in enumerate( New_Slice.Ranges() ) ]
                    New_Ranges = Adjusted_Ranges
                New_Ranges += [ ( min(Stack.keys()) , max(Stack.keys()) ) ]
            else :
                New_Data = dict({})
                New_Ranges = []
            return Young_Structure( New_Data , New_Ranges )

        if len(Input) == 1 :
            Input = Input[0]
            if isinstance( Input , Integer ) :
                return Get_Young_Structure_From_Integer( Input )

            elif isinstance( Input , ( list , tuple ) ) :
                Stack = dict({})
                for EntryCounter , Entry in enumerate( Input ) :
                    assert isinstance( Entry , ( int , Integer ) ) , \
                           'If the input is given as list or tuple, then the '+str(EntryCounter+1)+'-th entry need to be an integer.'
                    Stack.update({ EntryCounter : Get_Young_Structure_From_Integer(Entry) })
                return Stacking_Young_Structures( Stack )

            elif isinstance( Input , str ) :
                Input = Input.upper()
                RightCounts = Input.count('R')
                LeftCounts  = Input.count('L')
                DownCounts  = Input.count('D')
                UpCounts    = Input.count('U')
                assert len(Input) == sum([ RightCounts , LeftCounts , DownCounts , UpCounts ]) , \
                       'The only admissible letters in `Input` are R (right), L (left), D (down), and U (up).'
                assert RightCounts == LeftCounts , \
                       'The number of R-letters ('+str(RightCounts)+'x) and L-letters ('+str(LeftCounts)+'x) need to coincide.'
                assert DownCounts == UpCounts , \
                       'The number of D-letters ('+str(DownCounts)+'x) and U-letters ('+str(UpCounts)+'x) need to coincide.'
                return Get_Young_Structure_From_Binary_Description( Input )

            elif isinstance( Input , dict ) :
                for New_Coordinate , New_Slice in Input.items() :
                    assert isinstance( New_Coordinate , ( int , Integer ) ) , \
                           'If the input is a dictionary, then the keys need to be integers.'
                    assert isinstance( New_Slice , ( Young_Structure ) ) , \
                           'If the input is a dictionary, then the values need to be Young structures.'
                    return Stacking_Young_Structures( Input )

            elif isinstance( Input , Young_Structure ) :
                return Input

            else :
                raise ValueError('The single input is inappropriate.')

        elif len(Input) == 2 :
            if Input[0] == 'Empty' :
                Ranges = Input[1]
                return Get_Empty_Young_Structure( Ranges )

            else :
                raise ValueError('The two inputs are inappropriate.')

        else :
            raise ValueError('The input is inappropriate.')


    def Coordinates ( self ) -> "Variables" :
        """Returns the variable coordinates."""
        return var([ 'x'+str(Counter) for Counter in range(self.Rank()) ])


    def Data ( self ) -> dict[ tuple : int ] :
        """Returns the attribute ``_Data``."""
        return self._Data


    def Expand_Ranges ( self , New_Ranges ) -> "Young_Structure" :
        """Returns a Young structure with expanded ranges."""
        assert isinstance( New_Ranges , list ) , \
               'The input for `New_Ranges` need to be a list.'
        assert len(New_Ranges) == self.Rank() , \
               'The length of `New_Ranges` need to coincide with the rank of `self`.'
        YS = self
        for RangeCounter , New_Range in enumerate( New_Ranges ) :
            if New_Range == None :
                pass

            else :
                Old_Minimum , Old_Maximum = YS.Ranges()[RangeCounter]
                assert isinstance( New_Range , tuple ) , \
                       'The '+str(RangeCounter)+'-th entry need to be a tuple.'
                assert len(New_Range) == 2 , \
                       'The '+str(RangeCounter)+'-th range tuple need to be of length 2.'
                New_Minimum , New_Maximum = New_Range
                assert New_Minimum <= New_Maximum , \
                       'The new minimum need to be smaller than or equal to the new maximum.'
                assert New_Minimum <= Old_Minimum , \
                       'The new minimum need to be smaller than or equal to the old minimum.'
                assert Old_Maximum <= New_Maximum , \
                       'The old maximum need to be smaller than or equal to the new maximum.'
                YS._Ranges[RangeCounter] = New_Range

        return YS


    def Locus ( self , *Conditions ) -> "Young_Structure" :
        """Returns locus of Young structure with respect to given conditions."""
        for ConditionCounter , Condition in enumerate( Conditions , start=1 ) :
            assert isinstance( Condition , sage.symbolic.expression.Expression ) , \
                   'The input for the '+str(ConditionCounter)+'-th condition need to be an symbolic expression.'
        x = self.Coordinates()
        New_Data = dict({})
        for Coordinate , Value in self :
            Value_Satisfies_All_Conditions = True
            for ConditionCounter , Condition in enumerate( Conditions , start=1 ) :
                if bool( Condition({x[CoordinateCounter] : CoordinateEntry for CoordinateCounter , CoordinateEntry in enumerate( Coordinate ) }) ) :
                    pass
                else :
                    Value_Satisfies_All_Conditions = False
                    break
            if Value_Satisfies_All_Conditions : New_Data.update({ Coordinate : Value })
        New_Ranges = self.Ranges()
        return self.__class__( New_Data , New_Ranges )


    def Move_Origin ( self , Steps:list[ int ] ) -> "Young_Structure" :
        """Returns a Young structure with moved origin."""
        assert isinstance( Steps , list ) , \
               'The input for `Steps` need to be a list.'
        assert len(Steps) == self.Rank() , \
               'The length of the tuple `Steps` need to coincide with the rank of `self`.'
        for StepCounter , Step in enumerate( Steps ) :
            assert isinstance( Step , ( int , Integer ) ) , \
                   'The '+str(StepCounter)+'-th step need to be an integer.'
        New_Data = { tuple( vector( ZZ , Coordinate ) + vector( ZZ , Steps ) ) : Value for Coordinate , Value in self }
        New_Ranges = [ ( Minimum+Steps[RageCounter] , Maximum+Steps[RageCounter] ) for RageCounter , ( Minimum , Maximum ) in enumerate( self.Ranges() ) ]
        return self.__class__( New_Data , New_Ranges )


    def Ranges ( self ) -> list[ tuple[ int , int ] ] :
        """Returns the attribute ``_Ranges``."""
        return self._Ranges


    def Rank ( self ) -> int :
        """Returns the attribute ``_Rank``."""
        return self._Rank


    def Remove_Coordinate ( self , Index ) -> None :
        """Remove non-varying coordinates."""
        assert Index in range(self.Rank()) , \
               'The input for `Index` need to be in the range '+str(range(self.Rank()))+'-.'
        New_Data = dict({})
        for Coordinate , Value in self :
            Coordinate = tuple([ CoordinateEntry for CoordinateCounter , CoordinateEntry in enumerate(Coordinate) if CoordinateCounter != Index ])
            New_Data.update({ Coordinate : Value })
        New_Ranges = [ Range for RangeCounter , Range in enumerate( self.Ranges() ) if RangeCounter != Index ]
        self.__init__( New_Data , New_Ranges )


    def Remove_Vanishing_Values ( self ) -> "Young_Structure" :
        """Returns a Young structure without zero-values."""
        return self.__class__( { Coordinate : Value for Coordinate , Value in self if Value != 0 } , self.Ranges() )


    def Sizes ( self ) -> list[ int ] :
        """Returns the sizes per direction."""
        return [ Maximum-Minimum+1 for Minimum , Maximum in self.Ranges() ]


    def Support ( self , Output_Type :str ='Absolute' ) -> "Young_Structure" :
        "Returns the support."
        if   Output_Type in [ 'Absolute' , 'abs' ] :
            Data = { tuple(self.Rank()*[0]) : 0 }
            Data.update({ Coordinate : 1 for Coordinate , Value in self if Value != 0 })
            return self.__class__( Data , self.Ranges() )
        elif Output_Type in [ 'Relative' , 'rel' , 'Sign' , 'sgn' ] :
            return self.__class__( { Coordinate : sgn(Value) for Coordinate , Value in self } , self.Ranges() )
        else :
            raise ValueError('The input for `Output_Type` is inappropriate.')


    def Volumne ( self ) -> int :
        """Returns the sum of all entries."""
        return sum([ Value for Coordinate , Value in self ])




    # ToDo: NEED TO BE DONE.
    def __lt__ ( self , other ) -> bool :
        """Compares Young structures with respect to the <-operator."""
        raise ValueError('The method `less-than` is not yet implemented.')


    #def Compress ( self , Direction :int ) -> "Young_Structure" :
    #    assert isinstance( Direction , ( int , Integer ) ) , \
    #           'The input for `Direction` need to be an integer.'
    #    assert Direction in range(self.Rank()) , \
    #           'The value of `Direction` need to be in the range from 0 to '+str(self.Rank())+'.'

    #def Inflate ( self , Direction :int ) -> "Young_Structure" :
    #    pass

In [3]:
class Young_Diagram ( Young_Structure ) :

    # Synonym for the method `Twist`
    def __call__ ( self , Step :int =0 ) -> "Young_Diagram" :
        """Return a twisted Young diagram."""
        return self.Twist( Step )


    def __init__( self , Data :dict , Ranges :list or None =None ) -> None :
        super( Young_Diagram , self ).__init__( Data , Ranges )

        assert self.Rank() == 2 , \
               '`self` need to be of rank 2.'

        xRange , yRange = self.Ranges()
        xMinimum , xMaximum = xRange
        yMinimum , yMaximum = yRange
        assert xMinimum == 0 , \
               'The x-minimum need to be zero.'
        assert yMinimum == 0 , \
               'The y-minimum need to be zero.'

        for Coordinate , Value in self :
            xCoordinate , yCoordinate = Coordinate
            assert Value in [ 0 , 1 ] , \
                   'The value at coordinate '+str(Coordinate)+' need to be zero or one.'
            if 0 < xCoordinate :
                assert Value <= self[(xCoordinate-1,yCoordinate)] , \
                       'The value at '+str((xCoordinate-1,yCoordinate))+' need to be smaller than or equal to the value at '+str((xCoordinate,yCoordinate))+'.'
            if 0 < yCoordinate :
                assert Value <= self[(xCoordinate,yCoordinate-1)] , \
                       'The value at '+str((xCoordinate,yCoordinate-1))+' need to be smaller than or equal to the value at '+str((xCoordinate,yCoordinate))+'.'


    def __lshift__ ( self , Step :int =1 ) -> "Young_Diagram" :
        """Returns a shifted Young diagram (cyclic action)."""
        return self >> -1*Step


    # Synonym for the method `Shift`
    def __rshift__ ( self , Step :int =1 ) -> "Young_Diagram" :
        """Returns a shifted Young diagram (cyclic action)."""
        return self.Shift( Step )


    def Binary_Description ( self ) -> str :
        Width = self.Width()
        xPointer = Width
        Height = self.Height()
        yPointer = 0
        Description = ''
        while 0 < xPointer and yPointer < Height :
            if self[xPointer,yPointer] < self[xPointer-1,yPointer] :
                yPointer += 1
                Description += 'D'
            else :
                xPointer -= 1
                Description += 'L'
        if 0 < xPointer      : Description += xPointer*'L'
        if yPointer < Height : Description += (Height-yPointer)*'D'
        return Description


    def Complement ( self ) -> "Young_Diagram" :
        """Returns the complement of `self`."""
        Width = self.Width()
        Height = self.Height()
        return self.__class__.Constructor( [ Width-Entry for Entry in self.Usual_Description()[::-1] ] , (Width,Height) )


    def Column_Differences ( self ) -> list[int] :
        """Returns the differences s_{i}-s_{i+1} for i running through xRange"""
        return self.Conjugate().Row_Differences()


    def Column_Heights ( self ) -> list[ int ] :
        """Returns the height of each column."""
        return self.Conjugate().Row_Widths()


    def Column_Volumnes ( self ) -> list[ int ] :
        """Returns the volumne of each column."""
        return self.Conjugate().Row_Volumnes()


    def Columns ( self ) -> Iterator[ list[ int ] ] :
        """Returns an iterator running over the columns."""
        return self.Conjugate().Rows()


    def Conjugate ( self ) :
        """Returns the conjugated Young diagram."""
        return super( Young_Diagram , self ).Conjugate( Permutation( '(1,2)' ) )


    @staticmethod
    def Constructor ( Description :tuple[ int ] or str , Sizes :tuple[ int , int ] or None =None ) -> "Young_Diagram" :
        if Description in [ None , 'Empty' , [] , tuple([]) , dict({}) ] :
            MinimumWidth = 0
            MinimumHeight = 0
            Description = [ 0 ]

        elif isinstance( Description , ( list , tuple ) ) :
            MinimumWidth  = max([1] + list(Description))
            MinimumHeight = len(Description)
            Previous_Entry = +infinity
            for EntryCounter , Current_Entry in enumerate( Description ) :
                assert isinstance( Current_Entry , ( int , Integer ) ) , \
                       'The '+str(EntryCounter)+'-th entry need to be an integer.'
                assert Previous_Entry >= Current_Entry , \
                       'The '+str(EntryCounter)+'-th entry need to be smaller than or equal to the previous one.'
                Previous_Entry = Current_Entry

        elif isinstance( Description , str ) :
            if set(Description).issubset({ 'D' , 'L' }) :
                MinimumWidth  = Description.count('L')
                MinimumHeight = Description.count('D')
                Description = MinimumWidth*'R' + Description + MinimumHeight*'U'
            else :
                raise ValueError( 'The input for `Description` is given as string, then its letters must be from the set { D (down) , L (letft) }.' )

        else :
            raise ValueError( 'The input for `Description is inappropriate.`' )

        if Sizes == None : Sizes = ( MinimumWidth , MinimumHeight )
        Width , Height = Sizes
        assert MinimumWidth <= Width , \
               'The width need to be => '+str(MinimumWidth)+'.'
        assert MinimumHeight <= Height , \
               'The height need to be => '+str(MinimumHeight)+'.'

        YS = Young_Structure.Constructor( Description )
        YD = Young_Diagram( YS.Data() , YS.Ranges() )
        return YD.Expand_Sizes( Width , Height )


    def Cyclic_Orbit ( self ) -> Iterator[ "Young_Diagram" ] :
        """Runs through the cyclic orbit of `self`."""
        First_Object = self
        Current_Object = First_Object
        while True :
            yield Current_Object
            Current_Object = Current_Object >> 1
            if Current_Object == First_Object : break


    def Expand_Sizes ( self , New_Width :int , New_Height :int ) -> "Young_Diagram" :
        """Returns a Young diagram with expanded sizes."""
        assert isinstance( New_Width , ( int , Integer ) ) , \
               'The input for `New_Width` need to be an integer.'
        assert self.Width() <= New_Width , \
               'The new width need to be greater than or equal to the (old) width.'
        assert isinstance( New_Height , ( int , Integer ) ) , \
               'The input for `New_Height` need to be an integer.'
        assert self.Height() <= New_Height , \
               'The new height need to be greater than or equal to the (old) height.'
        return self.Expand_Ranges( [ ( 0 , New_Width-1 ) , ( 0 , New_Height-1 ) ] )


    def Height ( self ) -> int :
        """Returns the size in y-direction."""
        return self.Sizes()[1]


    def Is_Upper_Triangular ( self ) -> bool :
        """Tests if `self` is upper triangular"""
        Width  = self.Width()
        Height = self.Height()
        for xPos , Column_Height in enumerate( self.Column_Heights() ) :
            if not Column_Height <= floor( Height*(Width-xPos-1) / Width ) : return False
        return True


    def N ( self ) -> int :
        """Returns the sum of width and height."""
        return sum(self.Sizes())


    def Row_Differences ( self ) -> list[int] :
        """Returns the differences s_{i}-s_{i+1} for j running through yRange"""
        s = self.Row_Volumnes() + [ 0 ]
        return [ s[j]-s[j+1] for j in self.yRange() ]


    def Row_Widths ( self ) -> list[ int ] :
        """Returns the volumne of each row."""
        return [ sum(Row) for Row in self.Support().Rows() ]


    def Row_Volumnes ( self ) -> list[ int ] :
        """Returns the volumne of each row."""
        return [ sum(Row) for Row in self.Rows() ]


    def Rows ( self ) -> Iterator[ list[ int ] ] :
        """Returns an iterator running over the rows."""
        xRange = self.xRange()
        yRange = self.yRange()
        for yPos in yRange :
            yield [ self[xPos,yPos] for xPos in xRange ]


    def Shift ( self , Step :int =1 ) -> "Young_Diagram" :
        """Returns a shifted Young diagram (cyclic action)."""
        assert isinstance( Step , ( int , Integer ) ) , \
               'The input for `Step` need to be an integer.'
        Step = Step % self.N()
        if Step == 0 :
            return self
        else :
            Old_Binary_Description = self.Binary_Description()
            New_Binary_Description = Old_Binary_Description[-1] + Old_Binary_Description[ : -1 ]
            New_Step = Step-1
            return Young_Diagram.Constructor( New_Binary_Description ).Shift(New_Step)


    def Twist ( self , Step :int =1 ) -> "Young_Diagram" :
        """Returns the Young diagram after a twist-action."""
        assert isinstance( Step , ( int , Integer ) ) , \
               'The input for `Step` need to be an integer.'
        New_YD = Young_Diagram.Constructor( [ Entry+Step for Entry in self.Usual_Description() ] )
        New_Width  = max([ New_YD.Width()  , self.Width()  ])
        New_Height = max([ New_YD.Height() , self.Height() ])
        return New_YD.Expand_Sizes( New_Width , New_Height )


    # Synonym for the method `Row_Volumnes`.
    def Usual_Description ( self ) -> list[ int ] :
        """Returns the usual description of `self`."""
        return self.Row_Volumnes()


    def Width ( self ) -> int :
        """Returns the size in x-direction."""
        return self.Sizes()[0]


    def xRange ( self ) -> list[ int ] :
        """Returns the range in x-direction."""
        return [ 0 .. self.Width()-1 ]


    def yRange ( self ) -> list[ int ] :
        """Returns the range in y-direction."""
        return [ 0 .. self.Height()-1 ]


    @staticmethod
    def Lexicographically_Ordered ( Width :int , Height :int ) -> Iterator[ "Young_Diagrams" ] :
        assert isinstance( Height , ( int , Integer ) ) , \
               'The input for `Height` need to be an integer.'
        assert 1 <= Height , \
               'The integer Height need to be greater than or equal to 1.'
        assert isinstance( Width , ( int , Integer ) ) , \
               'The input for `Width` need to be an integer.'
        assert 1 <= Width , \
               'The integer Width need to be greater than or equal to 1.'
        Usual_Descriptions = [ list(Usual_Description) for Usual_Description in IntegerListsLex( max_part=Width , length=Height , max_slope=0 ) ]
        Usual_Descriptions.reverse()
        for Usual_Description in Usual_Descriptions :
            yield Young_Diagram.Constructor( Usual_Description , ( Width , Height ) )


    @staticmethod
    def Upper_Triangulars ( Width :int , Height :int ) -> Iterator[ "Young_Diagrams" ] :
        assert isinstance( Height , ( int , Integer ) ) , \
               'The input for `Height` need to be an integer.'
        assert 1 <= Height , \
               'The integer Height need to be greater than or equal to 1.'
        assert isinstance( Width , ( int , Integer ) ) , \
               'The input for `Width` need to be an integer.'
        assert 1 <= Width , \
               'The integer Width need to be greater than or equal to 1.'
        Ceiling = [ floor( Width*(Height-Counter)/Height ) for Counter in [ 1 .. Height ] ]
        Usual_Descriptions = [ list(Usual_Description) for Usual_Description in IntegerListsLex( max_part=Width , length=Height , ceiling=Ceiling , max_slope=0 ) ]
        Usual_Descriptions.reverse()
        for Usual_Description in Usual_Descriptions :
            yield Young_Diagram.Constructor( Usual_Description , ( Width , Height ) )


    @staticmethod
    def Minimal_Upper_Triangulars ( Width :int , Height :int ) -> Iterator[ "Young_Diagrams" ] :
        Orbits = []
        for YD in Young_Diagram.Upper_Triangulars( Width , Height ) :
            YD_Did_Already_Appear = False
            for Partial_Orbit in Orbits :
                if YD in Partial_Orbit :
                    YD_Did_Already_Appear = True
                    break
            if not YD_Did_Already_Appear :
                Orbit_Length = 0
                Partial_Orbit = []
                for Fellow in YD.Cyclic_Orbit() :
                    Orbit_Length += 1
                    if Fellow.Is_Upper_Triangular() : Partial_Orbit += [ Fellow ]
                Orbits += [ Partial_Orbit ]
                yield YD , Orbit_Length


NameError: name 'Young_Structure' is not defined

In [1318]:
#Tab = 3*' '
#for YD in Young_Diagram.Upper_Triangulars( 4 , 8 ) :
#    YD = YD.Conjugate()
#    print( 'Current YD:' )
#    print( YD )
#    print( 'with orbit length '+str(len([ 1 for Fellow in YD.Cyclic_Orbit() ]))+'.' )
#    #print( Tab , 'Orbit:' )
#    #for Counter , Fellow in enumerate( YD.Cyclic_Orbit() , start=1 ) :
#    #    print( Counter )
#    #    print( '\n'.join([ Tab+Line for Line in str(Fellow).split('\n') ]) )
#    #    print()